In [12]:
import math
from typing import Tuple
import pandas as pd
import numpy as np
import torch
from torch import nn, Tensor
import torch.nn.functional as F
from torch.nn import TransformerEncoder, TransformerEncoderLayer
from torch.utils.data import dataset
from itertools import product
import copy
import time

In [13]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [14]:
def generate_square_subsequent_mask(sz: int) -> Tensor:
    """Generates an upper-triangular matrix of -inf, with zeros on diag."""
    return torch.triu(torch.ones(sz, sz) * float('-inf'), diagonal=1)
class PositionalEncoding(nn.Module):

    def __init__(self, d_model: int, dropout: float = 0.1, max_len: int = 5000):
        super().__init__()
        self.dropout = nn.Dropout(p=dropout)

        position = torch.arange(max_len).unsqueeze(1)
        div_term = torch.exp(torch.arange(0, d_model, 2) * (-math.log(10000.0) / d_model))
        pe = torch.zeros(max_len, 1, d_model)
        pe[:, 0, 0::2] = torch.sin(position * div_term)
        pe[:, 0, 1::2] = torch.cos(position * div_term)
        self.register_buffer('pe', pe)

    def forward(self, x: Tensor) -> Tensor:
        """
        Args:
            x: Tensor, shape [seq_len, batch_size, embedding_dim]
        """
        x = x + self.pe[:x.size(0)]
        return self.dropout(x)
def standardization(data):
    mu = np.mean(data, axis=0)
    sigma = np.std(data, axis=0)
    return (data - mu) / sigma
def Cor_Loss_func(X,Y):
    the_coef=torch.corrcoef(torch.cat([X.reshape([1,-1]),Y.reshape([1,-1])]))[0,1]
    return -the_coef
def Cor_Loss(X,Y):
    cor=[]
    for i in range(X.shape[0]):
        cor.append(Cor_Loss_func(X[i,:],Y[i,:]).detach().numpy())
    return torch.tensor(np.mean(cor),requires_grad=True)

In [15]:
class TransformerModel(nn.Module):

    def __init__(self, ntoken: int, d_model: int, nhead: int, d_hid: int,
                 nlayers: int, dropout: float = 0.5):
        super().__init__()
        self.model_type = 'Transformer'
        self.pos_encoder = PositionalEncoding(d_model, dropout)
        encoder_layers = TransformerEncoderLayer(d_model, nhead, d_hid, dropout)
        self.transformer_encoder = TransformerEncoder(encoder_layers, nlayers)
        self.encoder = nn.Linear(ntoken,d_model)
        self.d_model = d_model
        self.decoder = nn.Linear(d_model, ntoken)


    def init_weights(self) -> None:
        initrange = 0.1
        self.encoder.weight.data.uniform_(-initrange, initrange)
        self.decoder.bias.data.zero_()
        self.decoder.weight.data.uniform_(-initrange, initrange)

    def forward(self, src: Tensor, src_mask: Tensor) -> Tensor:
        """
        Args:
            src: Tensor, shape [seq_len, batch_size]
            src_mask: Tensor, shape [seq_len, seq_len]
        Returns:
            output Tensor of shape [seq_len, batch_size, ntoken]
        """
        src=src.reshape(-1,433,1)
        src = self.encoder(src.cuda()) * math.sqrt(self.d_model)
        src = self.pos_encoder(src)
        output = self.transformer_encoder(src, src_mask)
        output = self.decoder(output)
        return output

In [16]:
data_csv=pd.read_csv("industry_ret_data_1500.csv")
dataset = data_csv.values
dataset=np.delete(dataset,0,axis=1)
dataset = dataset.astype('float32')
dataset=standardization(dataset)
vari=[]
# for i in range(433):
#     svar=np.var(np.array(dataset)[1000:1200,i])
#     print(i,svar)
#     vari.append(svar)
# print(np.mean(vari))
train_data = torch.from_numpy(np.array(dataset[:1000]))
val_data = torch.from_numpy(np.array(dataset[1000:1200]))
test_data = torch.from_numpy(np.array(dataset[1200:]))
epochs = 5
ntokens=1
bptt =24
def get_batch(source: Tensor, i: int) -> Tuple[Tensor, Tensor]:
    """
    Args:
        source: Tensor, shape [full_seq_len, batch_size]
        i: int
    Returns:
        tuple (data, target), where data has shape [seq_len, batch_size] and
        target has shape [seq_len * batch_size]
    """
    seq_len = min(bptt, len(source) - 1 - i)
    data = source[i:i+seq_len]
    target = source[i+1:i+1+seq_len].reshape(-1)
    return data, target

In [17]:
def train(model: nn.Module) -> None:
    model.train()  # turn on train mode
    total_loss = 0.
    log_interval = 200
    start_time = time.time()
    src_mask = generate_square_subsequent_mask(bptt).to(device)

    num_batches = len(train_data) // bptt
    for batch, i in enumerate(range(0, train_data.size(0) - 1, bptt)):
        data, targets = get_batch(train_data, i)
        #print(data,torch.reshape(targets.cpu(), data.size()))
        batch_size = data.size(0)
        if batch_size != bptt:  # only on last batch
            src_mask = src_mask[:batch_size, :batch_size]
        loss_function = torch.nn.MSELoss()  # 正确
        output = model(data, src_mask)
        output = torch.mean(output, 2)

        loss = loss_function(torch.reshape(output.cuda(), targets.size()), targets.cuda())
        cor=Cor_Loss(output.cpu(), torch.reshape(targets.cpu(), output.size()))

        optimizer.zero_grad()
        loss.backward()
        torch.nn.utils.clip_grad_norm_(model.parameters(), 0.5)
        optimizer.step()

        total_loss += loss.item()
        if batch % log_interval == 0 and batch > 0:
            lr = scheduler.get_last_lr()[0]
            ms_per_batch = (time.time() - start_time) * 1000 / log_interval
            cur_loss = total_loss / log_interval
            ppl = math.exp(cur_loss)
            print(f'| epoch {epoch:3d} | {batch:5d}/{num_batches:5d} batches | '
                  f'lr {lr:02.2f} | ms/batch {ms_per_batch:5.2f} | '
                  f'loss {cur_loss:5.2f} | ppl {ppl:8.2f} | 'f'cor {cor:.5f}')
            total_loss = 0
            start_time = time.time()
def evaluate(model: nn.Module, eval_data: Tensor) :
    model.eval()  # turn on evaluation mode
    total_loss = 0.
    total_var=0.
    src_mask = generate_square_subsequent_mask(bptt).to(device)
    with torch.no_grad():
        for i in range(0, eval_data.size(0) - 1, bptt):
            data, targets = get_batch(eval_data, i)
            batch_size = data.size(0)
            if batch_size != bptt:
                src_mask = src_mask[:batch_size, :batch_size]
            output = model(data, src_mask)
            output_flat = output.view(ntokens, -1)[0]

            loss_function = torch.nn.MSELoss()  # 正确
            output1 = torch.mean(output, 2)
            v_cor=Cor_Loss(output.cpu(), torch.reshape(targets.cpu(),output.size()))
            total_loss += batch_size * loss_function(output_flat.cuda(), targets.cuda()).item()
            total_var+=batch_size*np.var(np.array(targets))
            #print(targets,output_flat)

   # print("var:",total_var/(len(eval_data)-1))
    return total_loss / (len(eval_data) - 1),v_cor

In [18]:
parameters=dict(
    emsize=[8,32,64,128,256],
    d_hid=[8,32,64,128,256],
    nlayers=[2,4,6,8],
    nhead=[2,4,8],
    dropout=[0.2,0.5],
    lr=[0.1,0.01,0.005,0.001]
)
param_values=[v for v in parameters.values()]
param=list(product(*param_values))
print(param)

[(8, 8, 2, 2, 0.2, 0.1), (8, 8, 2, 2, 0.2, 0.01), (8, 8, 2, 2, 0.2, 0.005), (8, 8, 2, 2, 0.2, 0.001), (8, 8, 2, 2, 0.5, 0.1), (8, 8, 2, 2, 0.5, 0.01), (8, 8, 2, 2, 0.5, 0.005), (8, 8, 2, 2, 0.5, 0.001), (8, 8, 2, 4, 0.2, 0.1), (8, 8, 2, 4, 0.2, 0.01), (8, 8, 2, 4, 0.2, 0.005), (8, 8, 2, 4, 0.2, 0.001), (8, 8, 2, 4, 0.5, 0.1), (8, 8, 2, 4, 0.5, 0.01), (8, 8, 2, 4, 0.5, 0.005), (8, 8, 2, 4, 0.5, 0.001), (8, 8, 2, 8, 0.2, 0.1), (8, 8, 2, 8, 0.2, 0.01), (8, 8, 2, 8, 0.2, 0.005), (8, 8, 2, 8, 0.2, 0.001), (8, 8, 2, 8, 0.5, 0.1), (8, 8, 2, 8, 0.5, 0.01), (8, 8, 2, 8, 0.5, 0.005), (8, 8, 2, 8, 0.5, 0.001), (8, 8, 4, 2, 0.2, 0.1), (8, 8, 4, 2, 0.2, 0.01), (8, 8, 4, 2, 0.2, 0.005), (8, 8, 4, 2, 0.2, 0.001), (8, 8, 4, 2, 0.5, 0.1), (8, 8, 4, 2, 0.5, 0.01), (8, 8, 4, 2, 0.5, 0.005), (8, 8, 4, 2, 0.5, 0.001), (8, 8, 4, 4, 0.2, 0.1), (8, 8, 4, 4, 0.2, 0.01), (8, 8, 4, 4, 0.2, 0.005), (8, 8, 4, 4, 0.2, 0.001), (8, 8, 4, 4, 0.5, 0.1), (8, 8, 4, 4, 0.5, 0.01), (8, 8, 4, 4, 0.5, 0.005), (8, 8, 4, 4, 0.

In [19]:
test_mse=[]
for emsize, d_hid,nlayers, nhead, dropout,lr in param:
    print(emsize, d_hid,nlayers, nhead, dropout,lr)
    model = TransformerModel(ntokens, emsize, nhead, d_hid, nlayers, dropout).to(device)
    criterion = torch.nn.MSELoss()

    optimizer = torch.optim.SGD(model.parameters(), lr=lr)
    scheduler = torch.optim.lr_scheduler.StepLR(optimizer, 1.0, gamma=0.95)
    best_val_loss = float('inf')
    best_model = None
    for epoch in range(1, epochs + 1):
        epoch_start_time = time.time()
        train(model)
        val_loss,v_cor = evaluate(model, val_data)
        val_ppl = math.exp(val_loss)
        elapsed = time.time() - epoch_start_time
       # print('-' * 89)
       # print(f'| end of epoch {epoch:3d} | time: {elapsed:5.2f}s | '
         #     f'valid loss {val_loss:.5f} | valid ppl {val_ppl:8.2f}| 'f'V_cor {v_cor:.5f}')
        #print('-' * 89)

        if val_loss < best_val_loss:
            best_val_loss = val_loss
            best_model = copy.deepcopy(model)

        scheduler.step()


    test_loss,t_cor= evaluate(best_model, test_data)
    test_mse.append([emsize, d_hid,nlayers, nhead, dropout,lr,test_loss])
    test_ppl = math.exp(test_loss)
    print('=' * 89)
    print(f'| End of training | test loss {test_loss:.5f} | '
          f'test ppl {test_ppl:.5f}|'f'test cor {t_cor:.5f}')
    print('=' * 89)

8 8 2 2 0.2 0.1
| End of training | test loss 1.15884 | test ppl 3.18622|test cor 0.09621
8 8 2 2 0.2 0.01
| End of training | test loss 1.17225 | test ppl 3.22925|test cor 0.15771
8 8 2 2 0.2 0.005
| End of training | test loss 1.21274 | test ppl 3.36267|test cor -0.04338
8 8 2 2 0.2 0.001
| End of training | test loss 1.29595 | test ppl 3.65447|test cor -0.03868
8 8 2 2 0.5 0.1
| End of training | test loss 1.15868 | test ppl 3.18572|test cor 0.01304
8 8 2 2 0.5 0.01
| End of training | test loss 1.15953 | test ppl 3.18842|test cor 0.04286
8 8 2 2 0.5 0.005
| End of training | test loss 1.17703 | test ppl 3.24473|test cor -0.06827
8 8 2 2 0.5 0.001
| End of training | test loss 1.45594 | test ppl 4.28853|test cor -0.02964
8 8 2 4 0.2 0.1
| End of training | test loss 1.15651 | test ppl 3.17881|test cor 0.07304
8 8 2 4 0.2 0.01
| End of training | test loss 1.16498 | test ppl 3.20586|test cor -0.01077
8 8 2 4 0.2 0.005
| End of training | test loss 1.17073 | test ppl 3.22435|test cor 

| End of training | test loss 1.55656 | test ppl 4.74248|test cor -0.09546
8 8 4 4 0.2 0.1
| End of training | test loss 1.15603 | test ppl 3.17729|test cor 0.08083
8 8 4 4 0.2 0.01
| End of training | test loss 1.15564 | test ppl 3.17604|test cor 0.09604
8 8 4 4 0.2 0.005
| End of training | test loss 1.17473 | test ppl 3.23726|test cor -0.05403
8 8 4 4 0.2 0.001
| End of training | test loss 1.24998 | test ppl 3.49026|test cor 0.08382
8 8 4 4 0.5 0.1
| End of training | test loss 1.15976 | test ppl 3.18918|test cor -0.06313
8 8 4 4 0.5 0.01
| End of training | test loss 1.15753 | test ppl 3.18207|test cor 0.13344
8 8 4 4 0.5 0.005
| End of training | test loss 1.23204 | test ppl 3.42822|test cor -0.00958
8 8 4 4 0.5 0.001
| End of training | test loss 1.68803 | test ppl 5.40879|test cor 0.18123
8 8 4 8 0.2 0.1
| End of training | test loss 1.16872 | test ppl 3.21787|test cor -0.00723
8 8 4 8 0.2 0.01
| End of training | test loss 1.16356 | test ppl 3.20132|test cor 0.14096
8 8 4 8 0.

| End of training | test loss 1.16729 | test ppl 3.21328|test cor -0.02822
8 8 6 4 0.5 0.001
| End of training | test loss 1.17481 | test ppl 3.23752|test cor -0.21998
8 8 6 8 0.2 0.1
| End of training | test loss 1.15874 | test ppl 3.18590|test cor 0.04906
8 8 6 8 0.2 0.01
| End of training | test loss 1.16533 | test ppl 3.20699|test cor -0.04941
8 8 6 8 0.2 0.005
| End of training | test loss 1.16154 | test ppl 3.19484|test cor 0.03064
8 8 6 8 0.2 0.001
| End of training | test loss 1.24390 | test ppl 3.46911|test cor 0.06965
8 8 6 8 0.5 0.1
| End of training | test loss 1.16068 | test ppl 3.19212|test cor -0.08271
8 8 6 8 0.5 0.01
| End of training | test loss 1.16998 | test ppl 3.22192|test cor -0.01113
8 8 6 8 0.5 0.005
| End of training | test loss 1.17649 | test ppl 3.24299|test cor 0.04289
8 8 6 8 0.5 0.001
| End of training | test loss 1.19572 | test ppl 3.30595|test cor 0.10328
8 8 8 2 0.2 0.1
| End of training | test loss 1.16027 | test ppl 3.19079|test cor -0.16504
8 8 8 2 

| End of training | test loss 1.16232 | test ppl 3.19735|test cor -0.12906
8 8 8 8 0.5 0.005
| End of training | test loss 1.16831 | test ppl 3.21655|test cor -0.06509
8 8 8 8 0.5 0.001
| End of training | test loss 1.18867 | test ppl 3.28270|test cor -0.04290
8 32 2 2 0.2 0.1
| End of training | test loss 1.15600 | test ppl 3.17721|test cor 0.02757
8 32 2 2 0.2 0.01
| End of training | test loss 1.15980 | test ppl 3.18930|test cor -0.05136
8 32 2 2 0.2 0.005
| End of training | test loss 1.18879 | test ppl 3.28310|test cor -0.11545
8 32 2 2 0.2 0.001
| End of training | test loss 1.42982 | test ppl 4.17793|test cor -0.03265
8 32 2 2 0.5 0.1
| End of training | test loss 1.16058 | test ppl 3.19179|test cor -0.03515
8 32 2 2 0.5 0.01
| End of training | test loss 1.15835 | test ppl 3.18469|test cor 0.09649
8 32 2 2 0.5 0.005
| End of training | test loss 1.16053 | test ppl 3.19163|test cor 0.20939
8 32 2 2 0.5 0.001
| End of training | test loss 1.35955 | test ppl 3.89446|test cor -0.00

| End of training | test loss 1.16087 | test ppl 3.19271|test cor 0.04547
8 32 4 2 0.5 0.01
| End of training | test loss 1.16359 | test ppl 3.20142|test cor 0.02578
8 32 4 2 0.5 0.005
| End of training | test loss 1.16388 | test ppl 3.20235|test cor 0.02458
8 32 4 2 0.5 0.001
| End of training | test loss 1.20957 | test ppl 3.35203|test cor -0.01699
8 32 4 4 0.2 0.1
| End of training | test loss 1.15598 | test ppl 3.17713|test cor 0.06907
8 32 4 4 0.2 0.01
| End of training | test loss 1.16138 | test ppl 3.19434|test cor 0.00011
8 32 4 4 0.2 0.005
| End of training | test loss 1.16188 | test ppl 3.19593|test cor -0.15623
8 32 4 4 0.2 0.001
| End of training | test loss 1.24501 | test ppl 3.47296|test cor -0.14117
8 32 4 4 0.5 0.1
| End of training | test loss 1.15885 | test ppl 3.18628|test cor -0.02636
8 32 4 4 0.5 0.01
| End of training | test loss 1.15872 | test ppl 3.18586|test cor -0.01573
8 32 4 4 0.5 0.005
| End of training | test loss 1.20461 | test ppl 3.33547|test cor 0.2057

...

| End of training | test loss 1.17162 | test ppl 3.22722|test cor 0.10977
256 128 6 4 0.2 0.005
| End of training | test loss 1.16650 | test ppl 3.21074|test cor 0.17979
256 128 6 4 0.2 0.001
| End of training | test loss 1.16818 | test ppl 3.21613|test cor 0.15045
256 128 6 4 0.5 0.1
| End of training | test loss 1.16686 | test ppl 3.21189|test cor 0.12392
256 128 6 4 0.5 0.01
| End of training | test loss 1.16478 | test ppl 3.20522|test cor 0.19398
256 128 6 4 0.5 0.005
| End of training | test loss 1.18059 | test ppl 3.25629|test cor 0.01952
256 128 6 4 0.5 0.001
| End of training | test loss 1.27654 | test ppl 3.58421|test cor 0.14867
256 128 6 8 0.2 0.1
| End of training | test loss 1.18483 | test ppl 3.27014|test cor 0.12968
256 128 6 8 0.2 0.01
| End of training | test loss 1.16321 | test ppl 3.20019|test cor 0.15288
256 128 6 8 0.2 0.005
| End of training | test loss 1.17656 | test ppl 3.24320|test cor 0.14831
256 128 6 8 0.2 0.001
| End of training | test loss 1.21350 | test p

| End of training | test loss 1.20907 | test ppl 3.35038|test cor -0.21757
256 128 8 8 0.2 0.1
| End of training | test loss 1.19467 | test ppl 3.30245|test cor 0.23917
256 128 8 8 0.2 0.01
| End of training | test loss 1.16550 | test ppl 3.20752|test cor 0.12852
256 128 8 8 0.2 0.005
| End of training | test loss 1.17353 | test ppl 3.23338|test cor -0.15784
256 128 8 8 0.2 0.001
| End of training | test loss 1.19914 | test ppl 3.31728|test cor 0.14034
256 128 8 8 0.5 0.1
| End of training | test loss 1.19557 | test ppl 3.30545|test cor 0.03622
256 128 8 8 0.5 0.01
| End of training | test loss 1.17794 | test ppl 3.24769|test cor 0.21891
256 128 8 8 0.5 0.005
| End of training | test loss 1.16320 | test ppl 3.20015|test cor 0.03484
256 128 8 8 0.5 0.001
| End of training | test loss 1.19634 | test ppl 3.30800|test cor -0.18154
256 256 2 2 0.2 0.1
| End of training | test loss 1.21799 | test ppl 3.38040|test cor 0.04966
256 256 2 2 0.2 0.01
| End of training | test loss 1.15814 | test p

| End of training | test loss 1.15823 | test ppl 3.18428|test cor 0.09576
256 256 2 8 0.5 0.005
| End of training | test loss 1.16466 | test ppl 3.20482|test cor 0.01677
256 256 2 8 0.5 0.001
| End of training | test loss 1.23103 | test ppl 3.42474|test cor 0.17782
256 256 4 2 0.2 0.1
| End of training | test loss 1.16771 | test ppl 3.21462|test cor 0.19044
256 256 4 2 0.2 0.01
| End of training | test loss 1.16793 | test ppl 3.21532|test cor 0.10018
256 256 4 2 0.2 0.005
| End of training | test loss 1.16200 | test ppl 3.19633|test cor 0.09731
256 256 4 2 0.2 0.001
| End of training | test loss 1.17691 | test ppl 3.24435|test cor 0.08745
256 256 4 2 0.5 0.1
| End of training | test loss 1.26456 | test ppl 3.54155|test cor 0.14482
256 256 4 2 0.5 0.01
| End of training | test loss 1.15928 | test ppl 3.18762|test cor 0.01236
256 256 4 2 0.5 0.005
| End of training | test loss 1.16183 | test ppl 3.19577|test cor -0.07693
256 256 4 2 0.5 0.001
| End of training | test loss 1.24807 | test 

| End of training | test loss 1.17728 | test ppl 3.24554|test cor -0.08474
256 256 6 2 0.5 0.1
| End of training | test loss 1.17758 | test ppl 3.24652|test cor 0.16027
256 256 6 2 0.5 0.01
| End of training | test loss 1.16800 | test ppl 3.21555|test cor -0.06253
256 256 6 2 0.5 0.005
| End of training | test loss 1.16897 | test ppl 3.21867|test cor 0.04285
256 256 6 2 0.5 0.001
| End of training | test loss 1.25175 | test ppl 3.49646|test cor -0.12423
256 256 6 4 0.2 0.1
| End of training | test loss 1.17689 | test ppl 3.24426|test cor 0.16589
256 256 6 4 0.2 0.01
| End of training | test loss 1.16129 | test ppl 3.19406|test cor 0.22591
256 256 6 4 0.2 0.005
| End of training | test loss 1.16752 | test ppl 3.21401|test cor 0.16029
256 256 6 4 0.2 0.001
| End of training | test loss 1.22042 | test ppl 3.38860|test cor 0.03404
256 256 6 4 0.5 0.1
| End of training | test loss 1.29851 | test ppl 3.66382|test cor 0.20779
256 256 6 4 0.5 0.01
| End of training | test loss 1.16636 | test p

| End of training | test loss 1.17775 | test ppl 3.24706|test cor -0.01188
256 256 8 4 0.2 0.005
| End of training | test loss 1.17302 | test ppl 3.23172|test cor 0.15213
256 256 8 4 0.2 0.001
| End of training | test loss 1.16862 | test ppl 3.21755|test cor -0.17611
256 256 8 4 0.5 0.1
| End of training | test loss 1.17124 | test ppl 3.22598|test cor 0.12071
256 256 8 4 0.5 0.01
| End of training | test loss 1.16350 | test ppl 3.20113|test cor 0.09215
256 256 8 4 0.5 0.005
| End of training | test loss 1.19392 | test ppl 3.30000|test cor 0.15594
256 256 8 4 0.5 0.001
| End of training | test loss 1.20934 | test ppl 3.35129|test cor 0.13047
256 256 8 8 0.2 0.1
| End of training | test loss 1.17873 | test ppl 3.25024|test cor 0.21407
256 256 8 8 0.2 0.01
| End of training | test loss 1.15656 | test ppl 3.17898|test cor 0.12842
256 256 8 8 0.2 0.005
| End of training | test loss 1.16066 | test ppl 3.19205|test cor 0.04053
256 256 8 8 0.2 0.001
| End of training | test loss 1.18184 | test

In [22]:
print(test_mse[np.argmax(np.array(test_mse)[:,-1])])

[8, 8, 4, 2, 0.2, 0.001, 1.9330300315518245]


In [23]:
print(test_mse[np.argmin(np.array(test_mse)[:,-1])])

[8, 32, 2, 8, 0.2, 0.01, 1.1523562502638203]
